<a href="https://colab.research.google.com/github/shivams289/Projects/blob/main/Web_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import requests
import urllib
import pandas as pd
import numpy as np

In [ ]:
source = requests.get('https://www.forbes.com/business').content

In [ ]:
soup = BeautifulSoup(source, 'html5lib')

In [ ]:
print(soup.prettify())

In [ ]:


articles = soup.find_all("h3")


In [ ]:
len(articles)

26

In [ ]:
#For fetching The 1st page articles before pressing "More Articles"

news_dict = {'News_content':[],'Link':[], 'Title':[]}

for i in range(len(articles)):
    
  # Getting the link of the article
  if articles[i].find('a') is not None:
    link = articles[i].find('a')['href']
    news_dict['Link'].append(link)
    
    # Getting the title
    title = articles[i].find('a').get_text()
    news_dict['Title'].append(title)
    
    # Reading the content of the article using the link that we featched(it is divided in paragraphs)
    news_article = requests.get(link)
    article_content = news_article.content
    soup_article = BeautifulSoup(article_content, 'html5lib')
    body = soup_article.find_all('div', class_='article-body')
    
    if len(body)!=0:
      x = body[0].find_all('p')
      
      # Unifying the paragraphs
      list_paragraphs = []
      for p in range(len(x)):
          paragraph = x[p].get_text()
          list_paragraphs.append(paragraph)
          final_article = " ".join(list_paragraphs)
          
      news_dict['News_content'].append(final_article)

In [ ]:
#len(data['blocks']['items'])

In [ ]:
# new_link = data['blocks']['items'][-1]['url']
# new_id = data['blocks']['items'][-1]['id']
# new_date = data['blocks']['items'][-1]['date']
# print(new_link)
# print(new_id)
# print(new_date)

In [9]:
import time
import random

uas = ['Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246',
       'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36',
       'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9',
       'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36',
       'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:15.0) Gecko/20100101 Firefox/15.0.1',
       'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36',
       'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:92.0) Gecko/20100101 Firefox/92.0',
       'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.2 Safari/605.1.15']

#Fetching the content id of last article on first page
last_id = soup.find_all("article", class_='stream-item et-promoblock-removeable-item et-promoblock-star-item')[-1]['data-et-id']

##Fetching the date id of last article on first page
date = soup.find_all("div", class_='stream-item__date')[-1]['data-date']

#First page contains 26 articles
start = 26

ids = last_id
#Below is the URL which changes after we press "More Articles" button, 3 fields changes values, which we will be updating to make more calls
url_to_fetch_more_articls = 'https://www.forbes.com/simple-data/chansec/stream/?date='+ str(date) + '&start='+str(start)+'&ids='+str(ids)+'&limit=25&sourceValue=channel_1&swimLane=&specialSlot=&streamSourceType=channelsection'

fetch = requests.get(url_to_fetch_more_articls)
data = fetch.json()

set_of_ids = set()

count = 1
while len(data['blocks']['items'])==10:

  for i in range(len(data['blocks']['items'])):


    new_link = data['blocks']['items'][i]['url']
    new_title = data['blocks']['items'][i]['title']

    new_news_article = requests.get(new_link)
    new_article_content = new_news_article.content
    new_soup_article = BeautifulSoup(new_article_content, 'html5lib')
    new_body = new_soup_article.find_all('div', class_='article-body')

    if len(new_body)!=0:
      count = count +1
      news_dict['Link'].append(new_link)
      news_dict['Title'].append(new_title)

      y = new_body[0].find_all('p')

      new_list_paragraphs = []
      for p in range(len(y)):
        paragraph = y[p].get_text()
        new_list_paragraphs.append(paragraph)
        final_article = " ".join(new_list_paragraphs)

      news_dict['News_content'].append(final_article)
  if count%100 == 0:
    print(count)
  
  start = start + 10 # Every "More Article" button press adds 10 more articles
  ids = data['blocks']['items'][-1]['id'] #updating content id
  new_date = data['blocks']['items'][-1]['date'] #update date fetched from last article after pressing "More Articles button"
  if ids in set_of_ids:
    print("Error!! trying to fetch same page again")
    break
  set_of_ids.add(ids)
  url_to_fetch_more_articls = 'https://www.forbes.com/simple-data/chansec/stream/?date=' + str(new_date) + '&start='+str(start)+'&ids='+str(ids)+'&limit=25&sourceValue=channel_1&swimLane=&specialSlot=&streamSourceType=channelsection'

  #Taking random time between API calls to avoid getting blocked, consecutive and fast calls can get us blocked too soon.
  time.sleep(random.randint(1, 4))

#generating new request using updated url using last element of previous fetched data list
  fetch = requests.get(url_to_fetch_more_articls, headers={"User-Agent": random.choice(uas)})
  data = fetch.json()  


Error!! trying to fetch same page again


In [10]:
df = pd.DataFrame(news_dict)

In [11]:
df.shape

(733, 3)

In [12]:
len(set(df['Title']))

706

In [13]:
len(set(df['Link']))


706

In [14]:
df = df.drop_duplicates()
print(len(df))

706


In [15]:
df.head()

,News_content,Link,Title
0,China's visa and international travel restric...,https://www.forbes.com/sites/russellflannery/2...,Resumption Of Business Travel Tops Amcham Chin...
1,Katy (Awkwafina) and Shang-Chi (Simu Liu) in M...,https://www.forbes.com/sites/scottmendelson/20...,Box Office: ‘Shang-Chi’ Continues To Superhero...
2,Former President Donald Trump waves to cheerin...,https://www.forbes.com/sites/markjoyella/2021/...,Fox News Cuts Away From Trump’s 9/11 Visit Aft...
3,U.S. Supreme Court Justice Stephen Breyer dow...,https://www.forbes.com/sites/alisondurkee/2021...,Supreme Court Justice Stephen Breyer Dismisses...
4,Sen. Joe Manchin (D-W.Va.) on Sunday tore int...,https://www.forbes.com/sites/andrewsolender/20...,"Tensions Erupt As Manchin Slams AOC, Warns ‘Am..."


In [16]:
df.to_csv('sharmadata.csv', index=False)